### SVI+Mortality and label by percentile - process

In [1]:
# Load Libraries

import os
import warnings

import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
svi_path = '/home/h6x/git_projects/ornl-gnn-experiment/model_1/data/raw/SVI2018_US_county.gdb'
overdose_path = '/home/h6x/git_projects/ornl-gnn-experiment/model_1/data/raw/HepVu_County_Opioid_Indicators_05DEC22.xlsx'

Process mortality data

In [3]:
def preprocess_overdose_data(overdose_df):
    """Preprocess overdose data."""
    overdose_df['GEO ID'] = overdose_df['GEO ID'].astype(str)
    overdose_df['GEO ID'] = overdose_df['GEO ID'].apply(lambda x: x.zfill(5))
    return overdose_df

In [4]:
overdose_df = pd.read_excel(overdose_path)

In [5]:
overdose_df = preprocess_overdose_data(overdose_df)

In [6]:
overdose_df.head()

,GEO ID,State Abbreviation,County Name,Opioid Prescription Rate 2020,Narcotic Overdose Mortality Rate 2014,Narcotic Overdose Mortality Rate 2015,Narcotic Overdose Mortality Rate 2016,Narcotic Overdose Mortality Rate 2017,Narcotic Overdose Mortality Rate 2018,Narcotic Overdose Mortality Rate 2019,Narcotic Overdose Mortality Rate 2020
0,01001,AL,Autauga County,98.3,8.2,8.8,10.7,9.8,10.9,9.2,11.6
1,01003,AL,Baldwin County,65.0,18.0,20.0,16.6,15.1,14.9,14.5,27.4
2,01005,AL,Barbour County,22.8,4.4,4.5,5.7,5.8,5.2,5.7,7.6
3,01007,AL,Bibb County,24.8,17.2,16.6,22.6,21.7,23.1,19.4,27.3
4,01009,AL,Blount County,22.8,18.6,18.9,22.7,27.0,19.9,20.3,24.2


In [7]:
# Rename "Narcotic Overdose Mortality Rate 2018" as NOD_2018
overdose_df.rename(columns={'Narcotic Overdose Mortality Rate 2018': 'NOD_2018'}, inplace=True)

In [8]:
# if 'NOD_2018' values are less than 0, set them to 0
overdose_df['NOD_2018'] = overdose_df['NOD_2018'].apply(lambda x: 0 if x < 0 else x)

In [9]:
overdose_df['NOD_2018'].describe()

count    3221.000000
mean       17.102111
std        10.718242
min         0.000000
25%        10.000000
50%        14.800000
75%        21.700000
max       114.400000
Name: NOD_2018, dtype: float64

process SVI data

In [10]:
def preprocess_svi_data(us_svi, raw_variables):
    """Preprocess SVI data by removing invalid values and normalizing."""
    for variable in raw_variables:
        # if values are -999.00, set them to 0
        us_svi[variable] = us_svi[variable].apply(lambda x: 0 if x == -999.00 else x)
        # us_svi = us_svi[us_svi[variable] != -999.00]

    for var in raw_variables:
        max_val = us_svi[var].max()
        min_val = us_svi[var].min()
        us_svi[var] = (us_svi[var] - min_val) / (max_val - min_val)
    
    return us_svi

In [11]:
us_svi = gpd.read_file(svi_path)

In [12]:
us_svi.describe()

,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,...,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_Length,Shape_Area
count,3142.000000,3.142000e+03,3142.000000,3.142000e+03,3142.000000,3.142000e+03,3142.000000,3.142000e+03,3142.000000,3142.000000,...,3142.000000,3142.000000,3142.000000,3.142000e+03,3142.000000,3142.000000,3142.000000,3.142000e+03,3142.000000,3142.000000
mean,1124.319749,1.027699e+05,4.546467,4.340684e+04,123.292171,3.810634e+04,386.899427,1.408207e+04,884.373966,3025.519414,...,0.099618,0.493635,1.166136,9.469372e+03,610.241248,10.081954,1.722693,9.268843e+04,2.632185,0.351100
std,3614.348182,3.299077e+05,25.621201,1.269416e+05,117.823369,1.156989e+05,387.616188,4.943911e+04,1029.425965,11078.516810,...,0.299538,0.725523,17.958274,3.783075e+04,765.211388,5.099991,1.230198,2.842825e+05,3.588088,1.811359
min,2.046443,7.500000e+01,0.000000,7.000000e+01,8.000000,3.300000e+01,9.000000,-9.990000e+02,-999.000000,-999.000000,...,0.000000,0.000000,-999.000000,2.000000e+00,2.000000,1.700000,0.100000,6.600000e+01,0.126193,0.000550
25%,431.120769,1.094800e+04,0.000000,5.488000e+03,53.000000,4.231750e+03,174.000000,1.591750e+03,318.000000,255.250000,...,0.000000,0.000000,0.000000,9.755000e+02,213.000000,6.200000,0.900000,9.471500e+03,1.641198,0.117593
50%,616.479521,2.573600e+04,0.000000,1.246600e+04,87.000000,9.875000e+03,279.000000,3.989000e+03,599.000000,667.000000,...,0.000000,0.000000,1.000000,2.402500e+03,397.500000,9.200000,1.400000,2.275600e+04,1.993165,0.166887
75%,924.005428,6.720900e+04,0.000000,3.142000e+04,147.750000,2.602025e+04,456.000000,9.761500e+03,1046.000000,1905.250000,...,0.000000,1.000000,2.000000,6.099750e+03,705.750000,12.675000,2.200000,6.211300e+04,2.623161,0.249455
max,145573.832761,1.009805e+07,380.000000,3.524321e+06,1444.000000,3.306109e+06,5533.000000,1.589956e+06,15303.000000,357178.000000,...,1.000000,4.000000,11.000000,1.086657e+06,11938.000000,45.600000,13.600000,8.152241e+06,71.549539,74.257031


In [13]:
raw_variables = [
    'EP_POV', 'EP_UNEMP', 'EP_NOHSDP', 'EP_UNINSUR', 'EP_AGE65',
    'EP_AGE17', 'EP_DISABL', 'EP_SNGPNT', 'EP_LIMENG', 'EP_MINRTY', 'EP_MUNIT',
    'EP_MOBILE', 'EP_CROWD', 'EP_NOVEH', 'EP_GROUPQ'
]

In [14]:
# if raw variables values are -999.00, set them to 0 and min max scale
us_svi = preprocess_svi_data(us_svi, raw_variables)

In [15]:
us_svi.head()

,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,...,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_Length,Shape_Area,geometry
0,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,...,1,-999,4160,588,0.202733,1.5,32290,6.452810,1.536344,"MULTIPOLYGON (((-107.62554 36.56587, -107.6252..."
1,01,ALABAMA,AL,Autauga,01001,"Autauga County, Alabama",594.443459,55200,0,23315,...,0,0,3875,508,0.123007,0.9,37301,2.052740,0.150256,"MULTIPOLYGON (((-86.92120 32.65754, -86.92035 ..."
2,01,ALABAMA,AL,Blount,01009,"Blount County, Alabama",644.830460,57645,0,24222,...,0,0,6303,732,0.211845,1.3,40036,2.392326,0.164403,"MULTIPOLYGON (((-86.96336 33.85822, -86.95967 ..."
3,01,ALABAMA,AL,Butler,01013,"Butler County, Alabama",776.838201,20025,0,10026,...,0,0,2005,340,0.193622,1.7,17280,1.818327,0.191747,"MULTIPOLYGON (((-86.90894 31.96167, -86.87498 ..."
4,01,ALABAMA,AL,Calhoun,01015,"Calhoun County, Alabama",605.867251,115098,0,53682,...,0,0,10686,796,0.175399,0.7,117894,2.194795,0.154336,"MULTIPOLYGON (((-86.14622 33.70218, -86.14577 ..."


In [16]:
us_svi.shape

(3142, 126)

In [17]:
overdose_df.shape

(3221, 11)

In [18]:
merged_df = pd.merge(us_svi, overdose_df[['GEO ID', 'NOD_2018']],
                             left_on='FIPS', right_on='GEO ID', how='left')

In [19]:
merged_df.shape

(3142, 128)

In [20]:
merged_df.head()

,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,...,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_Length,Shape_Area,geometry,GEO ID,NOD_2018
0,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,...,4160,588,0.202733,1.5,32290,6.452810,1.536344,"MULTIPOLYGON (((-107.62554 36.56587, -107.6252...",35039,80.0
1,01,ALABAMA,AL,Autauga,01001,"Autauga County, Alabama",594.443459,55200,0,23315,...,3875,508,0.123007,0.9,37301,2.052740,0.150256,"MULTIPOLYGON (((-86.92120 32.65754, -86.92035 ...",01001,10.9
2,01,ALABAMA,AL,Blount,01009,"Blount County, Alabama",644.830460,57645,0,24222,...,6303,732,0.211845,1.3,40036,2.392326,0.164403,"MULTIPOLYGON (((-86.96336 33.85822, -86.95967 ...",01009,19.9
3,01,ALABAMA,AL,Butler,01013,"Butler County, Alabama",776.838201,20025,0,10026,...,2005,340,0.193622,1.7,17280,1.818327,0.191747,"MULTIPOLYGON (((-86.90894 31.96167, -86.87498 ...",01013,8.8
4,01,ALABAMA,AL,Calhoun,01015,"Calhoun County, Alabama",605.867251,115098,0,53682,...,10686,796,0.175399,0.7,117894,2.194795,0.154336,"MULTIPOLYGON (((-86.14622 33.70218, -86.14577 ...",01015,15.8


In [21]:
# create a new column as label and by looking at the NOD_2018 values, set the label as 1 if above 90th percentile, else 0
merged_df['label_90'] = merged_df['NOD_2018'].apply(lambda x: 1 if x > merged_df['NOD_2018'].quantile(0.90) else 0)

In [22]:
merged_df['label_80'] = merged_df['NOD_2018'].apply(lambda x: 1 if x > merged_df['NOD_2018'].quantile(0.80) else 0)

In [23]:
merged_df['label_95'] = merged_df['NOD_2018'].apply(lambda x: 1 if x > merged_df['NOD_2018'].quantile(0.95) else 0)

Adding GNN ID to the df

In [24]:
# add a GNN_ID column to the dataframe
merged_df['GNN_ID'] = range(len(merged_df))

In [25]:
merged_df.head()

,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,...,E_DAYPOP,Shape_Length,Shape_Area,geometry,GEO ID,NOD_2018,label_90,label_80,label_95,GNN_ID
0,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,...,32290,6.452810,1.536344,"MULTIPOLYGON (((-107.62554 36.56587, -107.6252...",35039,80.0,1,1,1,0
1,01,ALABAMA,AL,Autauga,01001,"Autauga County, Alabama",594.443459,55200,0,23315,...,37301,2.052740,0.150256,"MULTIPOLYGON (((-86.92120 32.65754, -86.92035 ...",01001,10.9,0,0,0,1
2,01,ALABAMA,AL,Blount,01009,"Blount County, Alabama",644.830460,57645,0,24222,...,40036,2.392326,0.164403,"MULTIPOLYGON (((-86.96336 33.85822, -86.95967 ...",01009,19.9,0,0,0,2
3,01,ALABAMA,AL,Butler,01013,"Butler County, Alabama",776.838201,20025,0,10026,...,17280,1.818327,0.191747,"MULTIPOLYGON (((-86.90894 31.96167, -86.87498 ...",01013,8.8,0,0,0,3
4,01,ALABAMA,AL,Calhoun,01015,"Calhoun County, Alabama",605.867251,115098,0,53682,...,117894,2.194795,0.154336,"MULTIPOLYGON (((-86.14622 33.70218, -86.14577 ...",01015,15.8,0,0,0,4


Save the df

In [26]:
output_dir = "/home/h6x/git_projects/ornl-gnn-experiment/model_1/data/processed"

In [27]:
gdf = gpd.GeoDataFrame(merged_df, geometry='geometry')

In [28]:
output_path = os.path.join(output_dir,'2018', f'svi_od_ranked_2018.shp')
os.makedirs(os.path.dirname(output_path), exist_ok=True)

In [29]:
gdf.to_file(output_path, driver='ESRI Shapefile')

/tmp/ipykernel_2371891/3225144930.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_path, driver='ESRI Shapefile')


In [31]:
# save the processed data as csv
output_path = os.path.join(output_dir,'2018', f'svi_od_ranked_2018.csv')
os.makedirs(os.path.dirname(output_path), exist_ok=True)

merged_df.to_csv(output_path, index=False)